In [5]:
import re
import requests
from bs4 import BeautifulSoup


In [6]:
bashUrl = 'https://tieba.baidu.com/f?kw=%E5%B0%8F%E8%B5%A2%E5%8D%A1%E8%B4%B7&ie=utf-8&tp=0&pn='
tieba = 'https://tieba.baidu.com/'

In [7]:
title = [] # 标题
author = [] # 作者
responser = [] # 回复人
content = [] # 内容
time_info = [] # 时间


In [8]:
# 解析回复
def parse_response_page(url):
    """
    :param url:
    :return:
    {1: {'content': None,
    'responser': '李金龙639',
    'time_info': '2018-07-20 07:49'},
   2: {'content': '            3期的按理说很快~',
    'responser': '再遇太难',
    'time_info': '2018-07-20 07:50'}}
    """
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, "lxml")
        datas = soup.findAll(attrs={'class': 'l_post l_post_bright j_l_post clearfix '})
        i = 1 # 每个贴下回复的条数
        replyDict = {} # 存储所有回复
        for data in datas:
            replyMessage = {} # 每一条回复

            # 回复
            responsers = data.find('a', {'class': 'p_author_name j_user_card'})
            for res in responsers.strings:
                responser.append(res.string)
                replyMessage['responser'] = res.string
            # 回复内容
            contents = data.find('div', {'class': 'd_post_content j_d_post_content '})
            content.append(contents.string)
            replyMessage['content'] = contents.string
            # 回复时间
            tail_info = data.findAll('span', {'class': 'tail-info'})
            time_info.append(tail_info[-1].string)
            replyMessage['time_info'] = tail_info[-1].string
            #print('第{}条回复已解析'.format(str(i)))
            #postMessage['response'] = replyMessage
            replyDict[i] = replyMessage
            i = i+1
    return replyDict # 返回回复的信息流

In [9]:
dataDict = {} # 保存全部数据字典
j = 1 # 总发帖的条数
for n in range(0, 500, 50): # 前10页的数据
    url = bashUrl + str(n)
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, "lxml")
        datas = soup.findAll(attrs={'class': ' j_thread_list clearfix'})
        for data in datas:
            postMessage = {}
            try:
                titles = data.find("a", {"class": "j_th_tit"})
                title.append(titles.string)
                postMessage['title'] = titles.string
                authors = data.find("a", {"class": "frs-author-name j_user_card "})
                author.append(authors.string)
                postMessage['author'] = authors.string
                # 解析回复项

                responseUrl = tieba + data.a['href']
                replyDict = parse_response_page(responseUrl)
            except AttributeError:
                continue
            postMessage['response'] = replyDict
            dataDict[j] = postMessage
            j = j + 1
        print('第{}页数据已解析'.format(n))
    else:
        print('网址{}登录失败'.format(url))

第0页数据已解析
第50页数据已解析
第100页数据已解析
第150页数据已解析
第200页数据已解析
第250页数据已解析
第300页数据已解析
第350页数据已解析
第400页数据已解析
第450页数据已解析


In [12]:
print(len(author))
print(len(title))
print(len(time_info))
len(dataDict)

476
480
2814


422

In [19]:
str(dataDict[422]['response'][2]['content']).strip()

'太贵，不敢申请'

In [14]:
dataDict

{1: {'title': '我是7月13号申请的，预计啥时候能到。等的心累。',
  'author': 'liwenlongyang6',
  'response': {1: {'responser': 'liwenlongyang6',
    'content': '            来个高手告知下，谢谢了。',
    'time_info': '2018-07-24 10:30'},
   2: {'responser': '你那样憔悴的美',
    'content': '            九号没到',
    'time_info': '2018-07-24 10:30'},
   3: {'responser': 'chaonannan6',
    'content': '            我也是13号的，哎',
    'time_info': '2018-07-24 10:38'},
   4: {'responser': 'vianna糖糖',
    'content': '            10天样子，除去周末休息，刚才客服的',
    'time_info': '2018-07-24 10:53'},
   5: {'responser': '诚信稳提',
    'content': '            同意13在等',
    'time_info': '2018-07-24 11:11'},
   6: {'responser': '孟伟7777',
    'content': '            我也13',
    'time_info': '2018-07-24 11:51'},
   7: {'responser': '一个不淑女的我',
    'content': '            十个工作日没到是不是可以取消了？我刚打电话说取消不了，利息很高的，下款后还扣手续费，',
    'time_info': '2018-07-24 12:20'}}},
 2: {'title': '10号的有谁到了没',
  'author': '无低押货款6',
  'response': {1: {'responser': '无低押货款6',
    'content': '    

### 保存部分数据（author 、title、response.content）到excel

In [20]:
saveAuthors = []
saveTitles = []
saveDate = []
saveContents = []
for i in range(1,len(dataDict)+1):
    saveAuthors.append(dataDict[i]['author'])
    saveTitles.append(dataDict[i]['title'])
    #saveDate.append(dataDict[i]['time_info'])
    responseDict = dataDict[i]['response']
    li = []
    for j in range(1,len(responseDict)+1):
        li.append(str(responseDict[j]['content']).strip())
    saveContents.append(li)

In [23]:
saveContents

[['来个高手告知下，谢谢了。',
  '九号没到',
  '我也是13号的，哎',
  '10天样子，除去周末休息，刚才客服的',
  '同意13在等',
  '我也13',
  '十个工作日没到是不是可以取消了？我刚打电话说取消不了，利息很高的，下款后还扣手续费，'],
 ['10号的有谁到了没', '没到 我6期的 你们呢', '12期10号没到', '靠网贷都是差钱的不放款害死人，还不让退。'],
 ['None'],
 ['怎么没人说话了', '没到', '没到'],
 ['发图秒删，啦啦啦',
  '9号不很多人到账了吗',
  '反正我没有到，发图秒删，我也是很绝望',
  '没到，我也9号',
  'None',
  '没有到',
  '没到。也是见了个鬼了',
  '今天还放吗'],
 ['9号没到的是多少期的啊？？', '被你们急死了 9号的没到 10号的在等'],
 ['139221的6期是不是快了？'],
 ['None', '我们九号的都还没到账，十一个工作日了', '不是把，9号的没到客服怎么说', '表示9号还没到'],
 ['None',
  '9号的，我去，，这倒闭就倒闭吧，，放款就取消好了。这么尴尬干嘛',
  '这破公司还不如倒闭呢。都十一个工作日了九号还不放款。违反契约精神',
  '**公司，本来借钱就是没钱了才借，这么拖不拖死人吗'],
 ['None', '目前服务已经恢复，请登陆卡贷APP重新操作，谢谢您对小赢卡贷的支持~'],
 ['',
  '我的直接额度都没有了，等这么多天',
  '我也是打开APp进不去，什么都点不进去，不知道是软件在升级还是什么的',
  '理财上现在是易贷到124了，我的是125估计今天有戏',
  '心脏病等出来了  哎 给人希望又让人失望'],
 ['感谢您关注小赢卡贷，以下是您可能关心的问题。',
  '我昨晚确认借款的，请问怎么现在还没有到账？',
  '有个问题，我想借钱，看你们月利率最低0.34%我怎么都算不出来这个数，到底是不是这个利率啊？？？',
  '请问怎么联系你呢，电话打不通呢',
  'None',
  '请问我信用卡25000额度，申请卡贷6万，过了3万，秒过的，放款几个钟头了，这也是已经过了？等到账就可以了是吗',
  '中午通过放款几点能到账啊',
  '打父

In [21]:
import xlwings as xw

In [30]:
wb = xw.Book('C:\\Users\\ms\\Desktop\\Tieba\\Tieba.xlsx')
detail_sheet = xw.sheets('Sheet1')
detail_sheet.range('A1').options(transpose=True).value = 'author'
detail_sheet.range('B1').options(transpose=True).value = 'title'
detail_sheet.range('C1').options(transpose=True).value = 'response'

detail_sheet.range('A2').options(transpose=True).value = saveAuthors
detail_sheet.range('B2').options(transpose=True).value = saveTitles
for i in range(len(saveContents)):
    detail_sheet.range('C'+str(i+2)).options(transpose=True).value = str(saveContents[i])